# 環境構築

In [1]:
!git clone https://github.com/akazdayo/MRTalk-GPT-SoVITS.git
%cd MRTalk-GPT-SoVITS

Cloning into 'MRTalk-GPT-SoVITS'...
remote: Enumerating objects: 4691, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4691 (delta 0), reused 1 (delta 0), pack-reused 4688 (from 1)
Receiving objects: 100% (4691/4691), 13.17 MiB | 18.20 MiB/s, done.
Resolving deltas: 100% (2699/2699), done.
/content/MRTalk-GPT-SoVITS


In [2]:
!uv sync -q
!uv add -q ffmpeg-python pyopenjtalk matplotlib-inline
!uv pip install pyngrok -q

In [ ]:
from google.colab import userdata
from pyngrok import ngrok

TOKEN = userdata.get("NGROK_TOKEN")
DOMAIN = userdata.get("NGROK_DOMAIN")
DATABASE_URL = userdata.get("DATABASE_URL")

ngrok.set_auth_token(TOKEN)

In [ ]:
with open(".env", "a") as f:
    f.write(f"DATABASE_URL={DATABASE_URL}")
!uv run prisma generate

In [ ]:
%%bash
cat <<EOF >  /content/MRTalk-GPT-SoVITS/GPT_SoVITS/configs/tts_infer.yaml
custom:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cuda
  is_half: true
  t2s_weights_path: GPT_SoVITS/pretrained_models/s1v3.ckpt
  version: v4
  vits_weights_path: GPT_SoVITS/pretrained_models/gsv-v4-pretrained/s2Gv4.pth
v1:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cpu
  is_half: false
  t2s_weights_path: GPT_SoVITS/pretrained_models/s1bert25hz-2kh-longer-epoch=68e-step=50232.ckpt
  version: v1
  vits_weights_path: GPT_SoVITS/pretrained_models/s2G488k.pth
v2:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cpu
  is_half: false
  t2s_weights_path: GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s1bert25hz-5kh-longer-epoch=12-step=369668.ckpt
  version: v2
  vits_weights_path: GPT_SoVITS/pretrained_models/gsv-v2final-pretrained/s2G2333k.pth
v3:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cpu
  is_half: false
  t2s_weights_path: GPT_SoVITS/pretrained_models/s1v3.ckpt
  version: v3
  vits_weights_path: GPT_SoVITS/pretrained_models/s2Gv3.pth
v4:
  bert_base_path: GPT_SoVITS/pretrained_models/chinese-roberta-wwm-ext-large
  cnhuhbert_base_path: GPT_SoVITS/pretrained_models/chinese-hubert-base
  device: cpu
  is_half: false
  t2s_weights_path: GPT_SoVITS/pretrained_models/s1v3.ckpt
  version: v4
  vits_weights_path: GPT_SoVITS/pretrained_models/gsv-v4-pretrained/s2Gv4.pth
EOF

In [ ]:
!rm ./GPT_SoVITS/pretrained_models/.gitignore
!git clone https://huggingface.co/lj1995/GPT-SoVITS ./GPT_SoVITS/pretrained_models/

# Launch Server
Google Colabのシークレット環境変数(左側のタブにあります)にNGROK_TOKEN, NGROK_DOMAIN, DATABASE_URLを追加する必要があります。  
https://ngrok.com/

In [ ]:
ngrok.connect(9000, domain=DOMAIN)

<NgrokTunnel: "https://inspired-curiously-kingfish.ngrok-free.app" -> "http://localhost:9000">

In [ ]:
!nohup uv run uvicorn client:app --port 9000 > server.log 2>&1 &

In [ ]:
!nohup uv run api_v2.py -a 127.0.0.1 -p 9880 -c GPT_SoVITS/configs/tts_infer.yaml > server2.log 2>&1 &